In [ ]:
import autoshop
import pandas as pd

In [ ]:
pd.set_option("display.max_rows", 1_000)

In [ ]:
df_food = autoshop.google.get_all_food()
df_shop = autoshop.google.get_shop()
df_food_conversion = autoshop.google.get_food_conversion()
df_tesco_food_map_raw = autoshop.google.get_tesco_food_map()

In [ ]:
df_tesco_food_map = (
    df_tesco_food_map_raw
    .dropna(subset=["amount"])
    .reset_index(drop=True)
    .assign(
        unit=lambda x: x["unit"].where(x["unit"] != "pack", ""),
        amountGram=lambda x: x[["amount", "unit"]].apply(
            lambda row: autoshop.unit.parse_to_grams(
                amount=row["amount"],
                unit=row["unit"], 
            ), 
            axis=1,
        ),
        amountMilliliter=lambda x: x[["amount", "unit"]].apply(
            lambda row: autoshop.unit.parse_to_milliliters(
                amount=row["amount"],
                unit=row["unit"],
            ), 
            axis=1,
        ),
    )
    [lambda x: x["amountMilliliter"].isna()]
    .drop(columns=["amountMilliliter"])
)

In [ ]:
df_food_shop = (
    df_shop
    .merge(df_food_conversion, how="left", on=["food", "unit"])
    .assign(
        amountGram=lambda x: x["amount"].where(x["unit"] == "g", x["amount"] * x["toGram"]),
        underlying=lambda x: x["amount"].astype(str) + " " + x["unit"],
    )
    .groupby(["food", "nameFood"])
    .aggregate(
        amountGram=pd.NamedAgg(column="amountGram", aggfunc=lambda x: x.sum(skipna=False)),
        underlying=pd.NamedAgg(column="underlying", aggfunc=lambda x: ", ".join(x)),
    )
    .reset_index()
)

display(df_food_shop)